In [5]:
extract_path = r"C:\Dataset"

In [7]:
import zipfile
import os

zip_path = r"C:\Users\BHUVANA VIJAYA\Downloads\Dataset.zip"
extract_path = r"C:\Dataset"  # much shorter path

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ ZIP extraction complete!")


✅ ZIP extraction complete!


In [17]:
import fitz
import re
import json
from pathlib import Path
from tqdm import tqdm
import platform

# Path to your extracted dataset
DATASET_PATH = Path(r"C:\Dataset")  # change to your folder

# Check if path exists
if not DATASET_PATH.exists():
    raise FileNotFoundError(f"Dataset path not found: {DATASET_PATH}")

author_texts = {}

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^A-Za-z0-9.,:;!?()\-\'"\s]', '', text)
    return text.strip().lower()

# Determine if Windows long path prefix is needed
WINDOWS = platform.system() == "Windows"

# Recursive scan for PDFs
all_pdf_files = list(DATASET_PATH.rglob("*.pdf"))
print(f"Found {len(all_pdf_files)} PDF files in dataset.")

for pdf_file in tqdm(all_pdf_files, desc="Processing PDFs"):
    author_name = pdf_file.parent.name  # folder containing PDF
    file_path = str(pdf_file)

    # Add long path prefix for Windows if needed
    if WINDOWS:
        file_path = "\\\\?\\" + file_path

    try:
        text = ""
        with fitz.open(file_path) as doc:
            for page in doc:
                text += page.get_text("text")
        clean = clean_text(text)
        if len(clean) > 200:  # ignore very short PDFs
            author_texts.setdefault(author_name, []).append(clean)
    except Exception as e:
        print(f"⚠️ Error reading {file_path}: {e}")

# Save JSON
output_path = DATASET_PATH / "author_texts.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(author_texts, f, ensure_ascii=False, indent=2)

print(f"✅ Extraction complete! Total authors processed: {len(author_texts)}")


Found 639 PDF files in dataset.


Processing PDFs:  22%|██▏       | 141/639 [00:08<00:31, 15.96it/s]

MuPDF error: library error: FT_New_Memory_Face(Times-Bold): unknown file format

MuPDF error: library error: FT_New_Memory_Face(Times-Bold): unknown file format



Processing PDFs: 100%|██████████| 639/639 [00:43<00:00, 14.59it/s]

✅ Extraction complete! Total authors processed: 71


In [18]:
import json
import os

# Path to the JSON file
json_path = r"C:\Dataset\author_texts.json"

# Check if file exists
if not os.path.exists(json_path):
    raise FileNotFoundError(f"JSON file not found at {json_path}")

# Load JSON
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Check if JSON is empty
if not data:
    print("⚠️ The JSON file is empty. No authors found.")
else:
    print(f"Total authors: {len(data)}")
    sample_authors = list(data.keys())[:5]
    print("Sample author names:", sample_authors)

    # Preview first 500 characters of the first paper of the first author
    first_author = sample_authors[0]
    if data[first_author]:
        print(f"\nPreview of first paper for {first_author}:\n")
        print(data[first_author][0][:500])
    else:
        print(f"No papers found for {first_author}")


Total authors: 71
Sample author names: ['Amit Saxena', 'Amita Jain', 'Animesh Chaturvedi', 'Ankita Jain', 'Arun Chauhan']

Preview of first paper for Amit Saxena:

a review of clustering techniques and developments amit saxena1, mukesh prasad2, akshansh gupta3, neha bharill4, om prakash patel4, aruna tiwari4, meng joo er5, weiping ding6, chin-teng lin2 1department of computer science  it, guru ghasidas vishwavidyalaya, bilaspur, india 2centre for artificial intelligence, university of technology sydney, sydney, australia 3school of computational and integrative sciences, jawaharlal nehru university, new delhi, india 4department of computer science and engi
